In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

In [ ]:
def get_dataset(d, n_train, n_test):
  x_train = np.random.randn(n_train, d)
  x_test = np.random.randn(n_test, d)
  mean_train = np.empty(d)
  norm_tab = np.empty(n_train)

  norm_train = (np.sum(x_train * x_train, axis = 1) ** 0.5).reshape(n_train, 1)
  x_train = (x_train / norm_train) * (d ** 0.5)
      
  norm_test = (np.sum(x_test * x_test, axis = 1) **0.5).reshape(n_test, 1)
  x_test = (x_test / norm_test) * (d**0.5)

  #On vérifie que notre data set soit uniforme (sur chaque dimension, le dataset a à peu près une moyenne de 0)
  #On se rapproche de plus en plus de 0 lorsqu'on augmente n_train, les données étant mieux réparties
  mean_train = np.sum(x_train, axis = 0) / n_train
      
  #On vérifie que chaque élément de notre dataset ait une norme de sqrt(d)
  norm_tab = np.sum(x_train * x_train, axis = 1) ** 0.5
      
  tau = 0.
  noise_level = tau * tau
  #On veut E(noise) == 0 et E(noise^2) == tau^2
  noise_train = np.random.randn(n_train, 1) * noise_level
  noise_test = np.random.randn(n_test, 1) * noise_level

  F1 = 1

  sample_params = np.random.randn(d, 1)
      
  norm = np.sum(sample_params * sample_params) #should be equal to F1^2
  sample_params = sample_params * (F1 / (norm**0.5))

  y_train = np.dot(x_train, sample_params) + noise_train
  y_test = np.dot(x_test, sample_params) + noise_test

  return x_train, y_train, x_test, y_test

In [ ]:
#Quelques fonctions d'activation :

class Sigmoid :
    @staticmethod
    def function(x):
        return 1/(1+np.exp(-x))
    
    @staticmethod
    def gradient(x):
        return Sigmoid.function(x) * Sigmoid.function(-x)
    
class Tanh :
    @staticmethod
    def function(x):
        return np.tanh(x)
        
    @staticmethod
    def gradient(x):
        t = Tanh.function(x)
        return 1-t**2
    
class Relu :
    @staticmethod
    def function(x):
        return np.maximum(x, 0)
    
    @staticmethod
    def gradient(x):
        return x >= 0

class Linear :
    @staticmethod
    def function(x):
        return x
    
    @staticmethod
    def gradient(x):
        return np.ones((x.shape[0], x.shape[1]))

In [ ]:
#Quelques Loss Functions :
class MSE:
    @staticmethod
    def loss(y_real, y_hat):
        return np.mean(np.sum((y_hat.T - y_real.T)**2, axis = 0))    
    
    @staticmethod
    def gradient(y_real, y_hat):
        return (2/y_real.shape[0]) * (y_hat - y_real)
    
class reg_MSE:
    def __init__(self, weights, lam):
        self.weights = weights
        self.lam = lam
        
    def loss(self, y_real, y_hat):
        loss = np.mean(np.sum((y_hat.T - y_real.T)**2, axis = 0))
        reg = (np.sum(np.square(self.weights[1])) + np.sum(np.square(self.weights[2])))*(self.lam * self.weights[1].shape[1]/self.weights[1].shape[0])
        return loss + reg
        
    def gradient(self, y_real, y_hat):
        loss_grad = (2/y_real.shape[0]) * (y_hat - y_real)
        reg_grad = (self.lam * self.weights[1].shape[1]/self.weights[1].shape[0]) * (np.sum(self.weights[1]) + np.sum(self.weights[2]))
        return loss_grad + reg_grad

In [ ]:
#L'architecture

class Network:
    def __init__(self, dimension_hidden, activation1, activation2):
        """
        dimension_hidden est le nombre de paramètres dans le hidden layer (N dans le papier de Mei et Montanari)
        activation1 est la fonction d'activation du hidden layer
        activation2 est la fonction d'activation de l'output layer
        """
        
        self.nb_layers = 3 #input, hidden, output
        self.dimensions = (d, dimension_hidden, 1)
                
        self.learning_rate = {}
        self.learning_rate[1] = None;  #learning rate du hidden layer
        self.learning_rate[2] = None;  #learning rate du output layer
        
        self.weights = {}
        self.bias = {}
                
        #on initialise les weights et les bias aléatoirement
        for i in range(1, self.nb_layers):
            self.weights[i] = np.random.randn(self.dimensions[i - 1], self.dimensions[i]) / np.sqrt(self.dimensions[i - 1])
            self.bias[i] = np.zeros(self.dimensions[i])
         
        self.activations = {}
        self.activations[2] = activation1
        self.activations[3] = activation2
        
    def forward_pass(self, x):
        """
        x est un vecteur de notre data
        
        return : z contient les paramètres avant que l'on applique l'activation function
        return : a contient les paramètres après que l'on applique l'activation function
        """
        z = {}
        a = {1:x} #l'input layer n'a pas d'activation function, a[1] est donc égal à x
        for i in range(1, self.nb_layers):
            z[i + 1] = np.dot(a[i], self.weights[i]) + self.bias[i] #Z = XW + b
            a[i + 1] = self.activations[i + 1].function(z[i + 1])
            
        return z, a
    
    def predict(self, x):
        _, a = self.forward_pass(x)
        return a[self.nb_layers]
    
    def back_propagation(self, z, a, y_real):
        y_hat = a[self.nb_layers]
        #On calcule delta et la dérivée partielle à l'output layer
        delta = self.loss_function.gradient(y_real, y_hat) * self.activations[self.nb_layers].gradient(y_hat)
        partial_deriv = np.dot(a[self.nb_layers - 1].T, delta)
     
        update_parameters = {
            self.nb_layers - 1: (partial_deriv, delta)
        }
        
        
        #On calcule delta et la dérivée partielle à l'hidden layer
        delta = np.dot(delta, self.weights[2].T) * self.activations[2].gradient(z[2])
        partial_deriv = np.dot(a[1].T, delta) 
        update_parameters[1] = (partial_deriv, delta)
            
        for k, v in update_parameters.items():
            self.update_weights_and_bias(k, v[0], v[1])
            
    def update_weights_and_bias(self, index, partial_deriv, delta):
        self.weights[index] -= self.learning_rate[index] * partial_deriv
        self.bias[index] -= self.learning_rate[index] * np.mean(delta, 0)

    def fit(self, x, y_real, x_test, y_test, loss, nb_iterations = 100, batch_size = 100, learning_rate1 = 0, learning_rate2 = 0.3):
        #On vérifie qu'on a autant de x que de y
        if not (x.shape[0] == y_real.shape[0]):
            raise Exception
            
        loss_tab = []
        
        self.loss_function = loss
        self.learning_rate[1] = learning_rate1
        self.learning_rate[2] = learning_rate2
        
        #We use batch gradient descent
        for i in range(nb_iterations):
            for j in range(x.shape[0] // batch_size):
                start = j * batch_size
                end = (j + 1) * batch_size
                z, a = self.forward_pass(x[start:end])
                self.back_propagation(z, a, y_real[start:end])
            _, a_train = self.forward_pass(x)
            _, a_test = self.forward_pass(x_test)
            loss_tab.append({
                "train_mse":  self.loss_function.loss(y_real, a_train[self.nb_layers]),
                "test_mse":  self.loss_function.loss(y_test, a_test[self.nb_layers]),
            })
            #if(i % 50) == 0:
                #print(f"Loss at Iteration {i} for first batch is {loss_tab[-1]}")
        return loss_tab

d = 100
n_train = 300
n_test = 300

x_train, y_train, x_test, y_test = get_dataset(d, n_train, n_test)
nn = Network(200, Linear, Linear)
config = dict(
        nb_iterations=30000, 
        batch_size=300, 
        learning_rate1=0, 
        learning_rate2=0.001
    )

loss_func = MSE #reg_MSE(nn.weights, 10**(-8))
history = nn.fit(x_train, y_train, x_test, y_test, loss_func, **config)

test_prediction = nn.predict(x_test)
print(test_prediction)

#print(y_test[0])

beta_calc = np.dot(np.dot(np.linalg.inv(np.dot(x_train.T, x_train)), x_train.T), y_train)
real_test_prediction = np.dot(beta_calc.T, x_test.T)
print("#########################")
print(real_test_prediction)
print(MSE.loss(test_prediction, real_test_prediction.T))

df = pd.DataFrame(history)
df.plot(grid=True)
mpl.ylim(-0.1, 3)
mpl.show()

In [ ]:
#With the formula

d = 1000
n_test = 300

gammas = np.arange(0.1, 3, 0.1)

l_f = []

for i, gam in enumerate(gammas):
    n_train = int((1/gam) * d)
    print(f"progress {100*i/len(gammas):.2f}%, n = {n_train}, gamma = {gam}")
    l0_f = []
    for _ in range(5):
        x_train, y_train, x_test, y_test = get_dataset(d, n_train, n_test)
        beta_calc = np.dot(np.dot(np.linalg.inv(np.dot(x_train.T, x_train)), x_train.T), y_train)
        test_prediction = np.dot(beta_calc.T, x_test.T)
        test_error = MSE.loss(y_test, test_prediction.T)
        l0_f.append(test_error)
        print(test_error)
    l0_f = np.array(l0_f)
    l_f.append({"mean": l0_f.mean(), "std": l0_f.std()})

In [ ]:
df = pd.DataFrame(l_f)
df.plot(grid=True)
mpl.ylim(-0.1, 2)
mpl.show()

In [ ]:
#With the neural network

d = 50
n_train = 150
n_test = 50

N_SPACE = np.arange(0.1, 2, 0.1)

l = []

for i, N in enumerate(N_SPACE):
    N = int(N*n_train)
    print(f"progress {100*i/len(N_SPACE):.2f}%, N = {N}")
    l0 = []
    for _ in range(10):
        x_train, y_train, x_test, y_test = get_dataset(d, n_train, n_test)

        nn = Network(N, Relu, Linear)
        config = dict(
            nb_iterations=500, 
            batch_size=n_train, 
            learning_rate1=0., 
            learning_rate2=0.01
        )
        loss_func = reg_MSE(nn.weights, 10**(-5))
        history = nn.fit(x_train, y_train, x_test, y_test, loss_func, **config)
        test_prediction = nn.predict(x_test)
        final_test_error = nn.loss_function.loss(y_test, test_prediction)
        l0.append(final_test_error)
        print(final_test_error)
    l0 = np.array(l0)
    l.append({"mean": l0.mean(), "std": l0.std()})


In [ ]:
df = pd.DataFrame(l)
df.plot(grid=True)
mpl.ylim(-0.1, 2)
mpl.show()

pd.DataFrame(l)